In [1]:
from workflow import create_graph
from langchain_ollama import ChatOllama

from agents.base_model import BaseModel
# from agents.hallucination_classifier import HallucinationClassifier

N = 3

model = ChatOllama(model="llama3.1")
base_models = [BaseModel(base_model=model, rank=i) for i in range(N)]

graph = create_graph(
    base_models=base_models,
    # hallu_classifier=HallucinationClassifier(base_model=model),
)

In [2]:
# Run
import pprint
inputs = {"question": "How to forget a girl?", "context": "Some relevant context about relationships."}
for output in graph.stream(inputs, {"recursion_limit": 100}):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value["responses"])

"Node 'BaseModel_1':"
'\n---\n'
"Node 'BaseModel_0':"
'\n---\n'
"Node 'BaseModel_2':"
'\n---\n'


InvalidUpdateError: At key 'question': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE

In [8]:
from langgraph.graph import Graph, START, END
from langgraph.types import Send
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage

# ===========================================
# 1️⃣ Define 3 separate Llama 3.1 8B instances
# ===========================================
llm_1 = ChatOllama(model="llama-3.1-8b", temperature=0.7)
llm_2 = ChatOllama(model="llama-3.1-8b", temperature=0.7)
llm_3 = ChatOllama(model="llama-3.1-8b", temperature=0.7)

def ask_llm(model, question: str):
    """Ask one LLM instance a question and return its response text."""
    response = model([HumanMessage(content=question)])
    return response.content.strip()

# ===========================================
# 2️⃣ Define graph nodes
# ===========================================
def start_node(state):
    q = state["question"]
    # fan out to all 3 llama_8b instances concurrently
    return [
        Send("llama_8b_1_node", {"q_1": q}),
        Send("llama_8b_2_node", {"q_2": q}),
        Send("llama_8b_3_node", {"q_3": q}),
    ]

def llama_8b_1_node(state):
    return {"ans_1": ask_llm(llm_1, state["q_1"])}

def llama_8b_2_node(state):
    return {"ans_2": ask_llm(llm_2, state["q_2"])}

def llama_8b_3_node(state):
    return {"ans_3": ask_llm(llm_3, state["q_3"])}

def aggregate_node(state):
    answers = [
        f"Llama 8B [1]: {state.get('ans_1')}",
        f"Llama 8B [2]: {state.get('ans_2')}",
        f"Llama 8B [3]: {state.get('ans_3')}",
    ]
    summary = "🧩 Aggregated responses from 3 Llama 3.1 8B instances:\n\n" + "\n\n".join(answers)
    return {"final_answer": summary}

# ===========================================
# 3️⃣ Build graph
# ===========================================
graph = Graph()
graph.add_node("start_node", start_node)
graph.add_node("llama_8b_1_node", llama_8b_1_node)
graph.add_node("llama_8b_2_node", llama_8b_2_node)
graph.add_node("llama_8b_3_node", llama_8b_3_node)
graph.add_node("aggregate_node", aggregate_node)

graph.add_edge(START, "start_node")
graph.add_edge("llama_8b_1_node", "aggregate_node")
graph.add_edge("llama_8b_2_node", "aggregate_node")
graph.add_edge("llama_8b_3_node", "aggregate_node")
graph.add_edge("aggregate_node", END)

graph = graph.compile()
# ===========================================
# 4️⃣ Run
# ===========================================
if __name__ == "__main__":
    import pprint

    inputs = {"question": "What are effective ways to overcome procrastination?"}

    print("⚙️ Streaming progress...\n")
    for output in graph.stream(inputs, {"recursion_limit": 100}):
        for key, value in output.items():
            pprint.pprint(f"Node '{key}': {value}")

    result = graph.invoke(inputs)
    print("\n✅ FINAL RESULT:\n")
    print(result)


⚙️ Streaming progress...

("Node 'start_node': [Send(node='llama_8b_1_node', arg={'q_1': 'What are "
 "effective ways to overcome procrastination?'}), Send(node='llama_8b_2_node', "
 "arg={'q_2': 'What are effective ways to overcome procrastination?'}), "
 "Send(node='llama_8b_3_node', arg={'q_3': 'What are effective ways to "
 "overcome procrastination?'})]")

✅ FINAL RESULT:

None


In [17]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END
from typing import Annotated, TypedDict
import pprint


# --- Define the shared state ---
class GraphState(TypedDict):
    question: str
    context: str
    responses: Annotated[dict, lambda x, y: {**x, **y}]  # merge rule


# --- Define model node ---
def chat_response(model_name: str):
    def _run(state: GraphState):
        model = ChatOllama(model=model_name)
        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                "You are a thoughtful and supportive assistant who provides emotionally intelligent and practical advice."
            ),
            (
                "user",
                (
                    "Given the context below, provide a calm, healthy, and actionable response.\n\n"
                    "Context: {context}\n\n"
                    "Question: {question}\n\n"
                    "Your response should focus on emotional well-being, mindset improvement, and practical steps."
                ),
            ),
        ])
        chain = prompt | model | StrOutputParser()

        answer = chain.invoke({
            "question": state["question"],
            "context": state["context"],
        })

        return {"responses": {model_name: answer}}

    return _run


# --- Build the graph ---
workflow = StateGraph(GraphState)
workflow.add_node("llama3.1", chat_response("llama3.1"))
workflow.add_node("deepseek-r1", chat_response("deepseek-r1"))

# Run both in parallel
workflow.add_edge(START, "llama3.1")
workflow.add_edge(START, "deepseek-r1")
workflow.add_edge("llama3.1", END)
workflow.add_edge("deepseek-r1", END)

# Compile
graph = workflow.compile()


# --- Run example ---
inputs: GraphState = {
    "question": "What is 1+1?",
    "context": "",
    "responses": {},
}

# Run graph — this returns the **final merged output**
final_state = graph.invoke(inputs)

# Pretty print cleanly
pprint.pprint(final_state)


{'context': '',
 'question': 'What is 1+1?',
 'responses': {'deepseek-r1': "Okay, let's approach this thoughtfully.\n"
                              '\n'
                              'The simple mathematical answer to "What is '
                              '1+1?" is 2. That\'s a basic fact.\n'
                              '\n'
                              "However, I understand you're asking for a "
                              'response focused on emotional well-being and '
                              "mindset, even for such a simple question. Let's "
                              'explore that perspective:\n'
                              '\n'
                              "1.  **Acknowledge the Question:** It's "
                              'perfectly normal to sometimes ask simple '
                              'questions or feel uncertain, even about basic '
                              "things. Don't judge yourself for this. "
                              'Accepting 